# Task 4.4.0
Task instruction:
Implement the DSPy Module: Create a DSPy module that uses the strategy you devise to generate a cooperative answer.

# Design plan

1) Define DSPy Signatures
Step 1 (Interpretation): history, current Q, context → history, current Q, context, student_goal, pragmatic_need
Step 2 (Retrieval): expanded_query, conversation_summary → refined_context
Step 3 (Answering): history, current Q, summary, goal, need, refined_context → cooperative_answer

2) Implement Retriever
Index HTML sources.
Retrieval cascade: BM25 (topic folder) → ColBERTv2/FAISS (fine passages).

3) Build Pipeline Forward Pass
Step 1 → Step 2 → Step 3.
Add loop if retrieval weak (multi-hop).

4) Prepare Data
Load PragmatiCQA train/val/test JSONL.
Convert each round to DSPy example.
Truncate long histories.

5) Train & Run
Few-shot or supervised training on train set.
Run pipeline on val/test sets.


# Step 1: configure the LLM API

In [33]:
# Configure the LLM Model
# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
import dspy
import os
from dotenv import load_dotenv
load_dotenv("../grok_key.ini",override=True)
lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])
dspy.configure(lm=lm)

# Step 2: Build the DSPY signatures and pipeline

IMPORTANT Note for Michael

Why we chose to enrich the model with the intermediary fields “student_goal” and “pragmatic_need”:

1) Capture the core idea: Cooperative answers rely on Theory of Mind (ToM) elements—understanding the user’s goals, intent, and needs. These fields allow the model to explicitly represent and reason about them.

2) Cost efficiency: If we instead generated “cooperative” questions and re-queried the context, the number of LLM calls and input tokens would increase significantly, raising costs. By focusing directly on the core idea, we can anticipate such questions more efficiently.

In [48]:
class ToMQuery(dspy.Signature):
    """
    Signature that enriches a user query with Theory of Mind insights - the asker's goals, intent and need
    """
    history = dspy.InputField(description="Previous conversation history of the user")
    current_question = dspy.InputField(description="The user's current question")
    context = dspy.InputField(description="Relevant context passages retrieved from the knowledge base")

    student_goal = dspy.OutputField(description="1-2 lines Summary of the user's learning goal or intent")
    pragmatic_need = dspy.OutputField(description="1-2 lines about the Underlying cooperative/pragmatic need inferred from the question")



class CooperativeAnswer(dspy.Signature):
    """
    Signature for generating a cooperative answer to the user's question.
    This module uses the enriched Theory of Mind information along with the
    retrieved context to produce a pragmatic and anticipatory answer that could also handle potential follow up questions.
    """
    history = dspy.InputField(
        description="Previous conversation history of the user, used for context and continuity"
    )
    current_question = dspy.InputField(
        description="The current question posed by the user"
    )
    
    context = dspy.InputField(
        description="Relevant context passages retrieved from the knowledge base or search module"
    )
    student_goal = dspy.InputField(
        description="Summary of the user's goal or interests, inferred from the conversation history"
    )
    pragmatic_need = dspy.InputField(
        description="Underlying cooperative or pragmatic need inferred from the user's question"
    )

    cooperative_answer = dspy.OutputField(
        description="Based on the input fields, generates a pragmatic and cooperative answer that anticipates potential follow-up questions"
    )

    # response= dspy.OutputField(
    # description="write something"
    # )


    # response = cooperative_answer

    # response = dspy.OutputField(
    #     description="Based on the input fields, generates a pragmatic and cooperative respomse that anticipates potential follow-up questions"
    # )




In [ ]:
# Switched from ChainOfThought to Predcit because of huge token usage
# Consider changing back to ChainOfThought in the future

class CooperativeQAPipeline(dspy.Module):
    """
    Pipeline that:
      1) Retrieves top context passages,
      2) Enriches the query with Theory-of-Mind (ToM) fields,
      3) Generates a cooperative answer using context + enrichment.
    """

    def __init__(self):
        super().__init__()
        # self.enrich_query = dspy.ChainOfThought(ToMQuery)
        # self.generate_answer = dspy.ChainOfThought(CooperativeAnswer)
        self.enrich_query = dspy.ChainOfThought(ToMQuery)
        self.generate_answer = dspy.ChainOfThought(CooperativeAnswer) # CooperativeAnswer

    def forward(self, examples: list[dspy.Example]):
        if not examples:
            return []

        # Step 1: Retrieve context
        print(f"Step 1: Retrieving top-k passages for {len(examples)} examples...")
        for ex in examples:
            ex.context = search(ex.current_question).passages
        print("Step 1 complete ✅")

        # Step 2: ToM enrichment
        print("Step 2: Running ToM enrichment...")
        tom_outputs = self.enrich_query.batch(examples)

        # Re-wrap enriched examples
        enriched_examples = []
        for idx, (ex, tom) in enumerate(zip(examples, tom_outputs), start=1):
            enriched_examples.append(
                dspy.Example(
                    history=ex.history,
                    current_question=ex.current_question,
                    context=ex.context,
                    student_goal=tom.student_goal,
                    pragmatic_need=tom.pragmatic_need,
                ).with_inputs("history", "current_question", "context", "student_goal", "pragmatic_need")
            )
            print(f"  → Enriched example {idx}/{len(examples)}")
        print("Step 2 complete ✅")

        # # Step 3: Generate cooperative answers
        # print("Step 3: Generating cooperative answers...")
        # answers = self.generate_answer.batch(enriched_examples)
        # for idx, _ in enumerate(answers, start=1):
        #     print(f"  → Answer generated for example {idx}/{len(enriched_examples)}")
        # print("Step 3 complete ✅")

        # return answers

        # Step 3: Generate cooperative answers
        print("Step 3: Generating cooperative answers...")
        answers = self.generate_answer.batch(enriched_examples)

        # Attach context to each answer
        for idx, (ans, ex) in enumerate(zip(answers, enriched_examples), start=1):
            ans.context = ex.context  # add the context to the returned answer
            print(f"  → Answer generated for example {idx}/{len(enriched_examples)}")
        print("Step 3 complete ✅")

        for ans in answers:
            ans.response = ans.cooperative_answer

        return answers


# STEP 3: set up the context retriever

In [4]:
# utility function from "pragmaticqa.ipynb" with cleanup
# Traverse a directory and read html files - extract text from the html files - and remove empty lines to reduce tokens usage laters

import os
from bs4 import BeautifulSoup
import re

def read_html_files_clean(directory):
    
    texts = []
    for filename in os.listdir(directory):
        if filename.endswith(".html"):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
                text = soup.get_text()
                # remove empty lines
                text = "\n".join([line.strip() for line in text.splitlines() if line.strip()])
                # optional: collapse multiple spaces
                text = re.sub(r'\s+', ' ', text)
                texts.append(text)
    return texts


In [5]:
# if the below code fails to run , run this:
# %pip install -U faiss-cpu  # or faiss-gpu if you have a GPU

In [6]:
# load the corpus , which is all the PragmatiCQA-sources html files

import os
folder_names = os.listdir("../PragmatiCQA-sources")
corpus = []
for folder in folder_names:
    texts = read_html_files_clean("../PragmatiCQA-sources/" + folder)
    corpus.extend(texts)
    print(f"added {len(texts)} files from {folder}")
    print("the total number of loaded files is: ", len(corpus))

added 449 files from 'Cats' Musical
the total number of loaded files is:  449
added 250 files from A Nightmare on Elm Street
the total number of loaded files is:  699
added 499 files from Arrowverse
the total number of loaded files is:  1198
added 492 files from Barney
the total number of loaded files is:  1690
added 498 files from Baseball
the total number of loaded files is:  2188
added 496 files from Batman
the total number of loaded files is:  2684
added 265 files from Big Nate
the total number of loaded files is:  2949
added 470 files from Bleach
the total number of loaded files is:  3419
added 310 files from Britney Spears
the total number of loaded files is:  3729
added 282 files from Detective Conan
the total number of loaded files is:  4011
added 499 files from Dinosaur
the total number of loaded files is:  4510
added 36 files from Doctor Who
the total number of loaded files is:  4546
added 322 files from Doom Patrol
the total number of loaded files is:  4868
added 61 files fr

In [7]:
# define the embedder and retriever , in similar manner to "rag.ipynb"
max_characters = 10000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

from sentence_transformers import SentenceTransformer

# Load an extremely efficient local model for retrieval
model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")

# Create an embedder using the model's encode method
embedder = dspy.Embedder(model.encode)

search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve) # set up the retriever function on the corpus

Training a 32-byte FAISS index with 338 partitions, based on 28573 x 1024-dim embeddings


# STEP 4: load the dataset which will be used by the pipeline

In [8]:
## "FOR THE FIRST QUESTION OF EACH CONVERSATION ONLY"

# load the dataset

# Load jsonl from dataset directory
import json
import os  
def read_data(filename, dataset_dir="../PragmatiCQA/data"):
    corpus = []
    with open(os.path.join(dataset_dir, filename), 'r') as f:
        for line in f:
            corpus.append(json.loads(line))
    return corpus

dataset_filename = "val.jsonl"
print("loading the dataset from: ", dataset_filename)
pcqa_test = read_data("val.jsonl")
print("Done loading the dataset.")



# unpack the questions into a single object
all_questions = [pcqa_test[i]['qas'][j]['q'] for i in range(len(pcqa_test)) for j in range(len(pcqa_test[i]['qas']))]
print(f"loaded: {len(all_questions)} questions")

# in task 4.4.1 , we need only the first question from the "val" set
first_questions_only = [pcqa_test[i]['qas'][0]['q'] for i in range(len(pcqa_test))]
first_questions_examples = [dspy.Example(current_question=question,
                                         history="",
                                         context="").with_inputs('current_question', 'history', 'context') for question in first_questions_only]



loading the dataset from:  val.jsonl
Done loading the dataset.
loaded: 1526 questions


# Step 5: run the pipeline on the dataset

In [ ]:
rag = CooperativeQAPipeline()  # create once
preds = rag(first_questions_examples)
print(preds)

Step 1: Retrieving top-3 passages for 15 examples...
Step 1 complete ✅
Step 2: Running ToM enrichment...
Processed 12 / 15 examples:  80%|████████  | 12/15 [00:18<00:03,  1.13s/it]

2025/08/22 19:24:11 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/08/22 19:24:11 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/22 19:24:13 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/08/22 19:24:13 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/22

Processed 15 / 15 examples: 100%|██████████| 15/15 [00:56<00:00,  3.77s/it]
  → Enriched example 1/15
  → Enriched example 2/15
  → Enriched example 3/15
  → Enriched example 4/15
  → Enriched example 5/15
  → Enriched example 6/15
  → Enriched example 7/15
  → Enriched example 8/15
  → Enriched example 9/15
  → Enriched example 10/15
  → Enriched example 11/15
  → Enriched example 12/15
  → Enriched example 13/15
  → Enriched example 14/15
  → Enriched example 15/15
Step 2 complete ✅
Step 3: Generating cooperative answers...
Processed 13 / 15 examples:  87%|████████▋ | 13/15 [00:19<00:02,  1.05s/it]

2025/08/22 19:25:01 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/08/22 19:25:01 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/22 19:25:07 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=4000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/08/22 19:25:07 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Processed 15 / 15 examples: 100%|██████████| 15/15 [00:52<00:00,  3.52s/it]
  → Answer generated for example 1/15
  → Answer generated for example 2/15
  → Answer generated for example 3/15
  → Answer generated for example 4/15
  → Answer generated for example 5/15
  → Answer generated for example 6/15
  → Answer generated for example 7/15
  → Answer generated for example 8/15
  → Answer generated for example 9/15
  → Answer generated for example 10/15
  → Answer generated for example 11/15
  → Answer generated for example 12/15
  → Answer generated for example 13/15
  → Answer generated for example 14/15
  → Answer generated for example 15/15
Step 3 complete ✅
[Prediction(
    reasoning="Based on the provided context, which details Freddy Krueger's origins, characteristics, and appearances in the A Nightmare on Elm Street series, I will craft a response that aligns with the user's goal of learning about his identity, origins, and key traits in a non-spoilery manner. The history is emp

# TASK 4.4.1
Task Instruction: Perform the same evaluation as in 4.3 on the first questions in each conversation and compare the results of your model with the one in 4.3 based on the traditional text-to-text transformer.

# Note for Michael:
In task 4.3 , we are requested to compute 3 different configurations because "the model depends on the accuracy of the retriever module".
But in the LLM it is not the case: it uses a combination of artifacts including the chat history and the pretraining knowledge. so we will compare just to the "retrieved answer".


# Another Note for Michael:

the class dspy.Evaluate.SemanticF1 directly calculates the F1 score , which is harmonic mean of precision and recall.
As you can see in the source code here - https://dspy.ai/api/evaluation/SemanticF1/#dspy.evaluate.SemanticF1.forward - 
it is calculated internally and we as users cannot alter it and return just the precision or recall without changing the "forward" source code.
So we will use just the F1 score.

In [ ]:
from dspy.evaluate import SemanticF1

metric = SemanticF1(decompositional=True)

examples = [
    dspy.Example(
        question=pcqa_test[i]['qas'][0]['q'],
        response=pcqa_test[i]['qas'][0]['a']) 
        for i in range(len(preds))]


# Compute the metric score for each prediction.
scores = []
for i in range(len(preds)):
    print(examples[i])
    print(preds[i])
    score = metric(examples[i], preds[i])
    print(score)
    scores.append(score)


# calculate the average F1 score and standard deviation , and provide a report
import numpy as np

np_scores = np.array(scores)
sd = np.std(np_scores)
mean = np.mean(np_scores)



@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

the total number of samples is:  15
the average F1 score is:  0.30277267380498735
the F1 score standard deviation is:  0.2309696366224379

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


In [ ]:
#############################################################
########## THE ANSWER TO 4.4.1 IS HERE BELOW !!! ############
#############################################################

print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n")
print("the total number of samples is: ", len(np_scores))
print("the average F1 score is: ", mean)
print("the F1 score standard deviation is: ", sd)
print("\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

# Insights:

1) The model achieves higher scores when the question is specific and contains named entities as well as unambiguous nouns and verbs. For example: “What is Batmans real name?” (score = 0.75)

2) This specificity provides a double advantage: it allows the retriever to fetch more relevant context, and it helps the pipeline rely effectively on the context in situations where there is no prior conversation history, which is the case in this experiment.

3) Conversely, the model performs poorly and can even score zero when the question is ambiguous or amorphous, for example: “What is the movie about?” (score = 0.0)
In such cases, the retriever often returns weak or unrelated context, and without previous chat history,the generated answer is largely irrelevant and is based on knowledge from the LLM's pretraining or on the irrelevant context.

4) these findings highlight the significant importance of the context , including both relevant documents and chat history , for cooperative answering.